# 第四讲：牛顿法

回到上一讲介绍的逻辑回归：在逻辑回归中，我们用S型函数作为$g(z)$，使用梯度上升的更新规则求参数$\theta$的最大似然估计。下面来介绍另一种方法：

## 7. 牛顿法

我们先回忆一下牛顿法找函数值零点：设函数$f:\ \mathbb{R}\to\mathbb{R}$，我们希望找到使$f(\theta)=0$的$\theta\in\mathbb{R}$值，牛顿法提供下面的更新规则：

$$\theta:=\theta-\frac{f(\theta)}{f'(\theta)}$$

该算法的大概流程为：先做对$\theta$做一次猜测$\theta=\theta_1$，然后过$f(\theta_1)$做$f$的切线，切线交横轴于点$\theta_2$（也就是切线函数值的零点），此时更新$\theta=\theta_2$，再过$f(\theta_2)$做$f$的切线，这样$\theta$就可以逼近使函数值为零的$\theta$取值，下面是牛顿法图解：

<img src="./resource/chapter04_image01.png" width="800" alt="" align=center />

在最左的图中为函数$f$的图像，我们希望找到零$f(\theta)=0$的点，从图中看这个点大约取在$1.3$附近。假设我们初始化算法时猜测$\theta^{(0)}=4.5$，牛顿法会计算出函数$f$在$(4.5,\ f(4.5))$点的切线并解出切线的零点（即切线与$y=0$的焦点，如中图所示）。这就是第二次猜测的起点，在图中$\theta^{(1)}$大约在$2.8$附近。而最右侧的图像显示了再一次迭代的结果，这使得$\theta^{(2)}$取到了$1.8$附近。在几轮迭代之后，$\theta$会快速向$1.3$附近收敛。

根据导数（切线斜率）的定义有$f'\left(\theta^{(0)}\right)=\frac{f\left(\theta^{(0)}\right)}{\Delta}$，于是$\Delta=\frac{f\left(\theta^{(0)}\right)}{f'\left(\theta^{(0)}\right)}$，所以$\theta^{(1)}=\theta^{(0)}-\Delta=\theta^{(0)}-\frac{f\left(\theta^{(0)}\right)}{f'\left(\theta^{(0)}\right)}$。则对于一般情况有$\theta^{(t+1)}=\theta^{(t)}-\frac{f\left(\theta^{(t)}\right)}{f'\left(\theta^{(t)}\right)}$。

牛顿法给了我们一个求$f(\theta)=0$的方法，我们也可以利用它来求函数$\mathscr{l}(\theta)$的最大值。函数的最大值点也就是$\mathscr{l}(\theta)$一阶倒数为零的点，所以只需要替换上面方法中的$f(\theta)=\mathscr{l}'(\theta)$即可，我们可以用牛顿方法，按照下面的更新规则求出函数最大值：

$$\theta:=\theta-\frac{\mathscr{l}'(\theta)}{\mathscr{l}''(\theta)}$$

上一讲中，我们在推导逻辑回归的最大值时，使用向量记法的$\theta$，我们将这一标记法应用到牛顿法中，在这种多维条件下一般化的牛顿法（也称作Newton-Raphson方法）为：

$$\theta:=\theta-H^{-1}\nabla_\theta\mathscr{l}(\theta)$$

* 式中的$\nabla_\theta\mathscr{l}(\theta)$仍然表示$\mathscr{l}(\theta)$关于向量$\theta$的每一个分量$\theta_i$的偏导数；
* $H$是一个$n$阶方阵（如果算上截距项应该是$n+1$阶方阵），称为**海森矩阵（Hessian matrix）**，为了便于记忆可以当做是一阶导数（向量）除以二阶导数（乘以方阵的逆），方阵的定义为：

$$H_{ij}=\frac{\partial^2\mathscr{l}(\theta)}{\partial\theta_i\partial\theta_j}$$

相比于（批量）梯度下降法，牛顿法收敛速度通常更快，且逼近最小值时需要的迭代次数更少（牛顿收敛是一个二阶收敛（quadratic convergence））。然而牛顿法单次迭代的代价通常比梯度下降法更大，因为它需要计算整个$n$阶方阵再求逆。通常情况下，在$n$不是很大的条件下，牛顿法总体上更快。当我们使用牛顿法求逻辑回归中对数化似然函数$\mathscr{l}(\theta)$的最大值时，这个算法也被称作**Fisher scoring**。

# 第三部分：一般线性模型（Generalized Linear Models）

到目前为止，我们了解了回归问题、分类问题：在回归问题中我们设$y\mid x;\theta\sim\mathcal{N}\left(\mu,\sigma^2\right)$，而在分类问题中我们设$y\mid x;\theta\sim\mathrm{Bernoulli}(\phi)$，对关于$x$和$\theta$的函数给出合理的$\mu$和$\phi$。我们会在这一章节看到，这两种假设都是一族算法模型中的特例，这一族算法模型就是我们曾经提到的一般线性模型（GLMs）。我们还将了解别的GLM家族中的模型如何推导、应用于其他分类、回归问题中。

## 8. 指数族（exponential family）

我们从定义指数族分布开始介绍GLMs，指数族的一类分布可以写为如下形式：

$$p(y;\eta)=b(y)\exp\left(\eta^TT(y)-a(\eta)\right)\tag{1}$$

* 式中的$\eta$称为分布的**自然参数（natural parameter）**或**规范参数（canonical parameter）**；
* $T(y)$称为**充分统计量（sufficient statistic）**（对于我们涉及的分布，通常$T(y)=y$）；
* $a(\eta)$称为**对数配分函数（log partition function）**
* $e^{-a(\eta)}$这个量起到标准化常量的作用，它使得$p(y;\eta)$求和或对$y$求积分的结果等于$1$。

如果选定了$T,\ a,\ b$，就能够定义出由$\eta$参数化的一族（或一个集合）分布，之后，我们通过改变$\eta$就能获得这一族分布中不同的分布函数。

可以看出伯努利分布和高斯分布都是指数族分布中的特例。期望为$\phi$的伯努利分布写作$\mathrm{Bernoulli}(\phi)$，分布$y\in\{0,\ 1\}$上取值，所以$p(y=1;\phi)=p;\ p(y=0;\phi)=1-\phi$，我们改变$\phi$则会得到有不同期望值的伯努利分布。可以看出这一类由$\phi$确定的伯努利分布就在指数族分布中，即将$(1)$式中的$T,\ a,\ b$确定下来后，式子就成为了伯努利分布类型的表达式。

我们将伯努利分布写为：

$$\begin{align}p(y;\phi)&=\phi^y(1-\phi)^{1-y}\\&=\exp(y\log\phi+(1-y)\log(1-\phi))\\&=\exp\left(\left(\log\left(\frac{\phi}{1-\phi}\right)\right)y+log(1-\phi)\right)\end{align}$$

因此，自然参数为$\mu=\log\frac{\phi}{1-\phi}$，有趣的是，如果我们反过来用从$\mu$的表达式中解出$\phi$，则有$\phi=\frac{1}{1+e^{-\eta}}$，这就是逻辑回归中使用的S型函数族！当我们用GLM的形式表达逻辑回归时，还将再次看到这一现象。为了将伯努利分布完全用指数族分布的形式表达，我们给出其他指数族分布的参数：

$$\begin{align}T(y)&=y\\a(\mu)&=-\log(1-\phi)\\&=\log(1+e^{\eta})\\b(y)&=1\end{align}$$

上面的参数表明，伯努利分布在选取适当的参数$T,\ a,\ b$后，就可以写为$(1)$式的形式。

接下来我们考虑高斯分布。回忆由概率模型推导线性回归的过程中，我们发现方差$\sigma^2$的取值并不影响我们最终得到的参数$\theta$和假设函数$h_\theta(x)$，因此我们即使选择任意的$\sigma^2$也不会对分布的表示造成任何影响。为了便于求导，我们令$\sigma^2=1$。（即使我们将$\sigma^2$以变量的形式留在概率密度函数中，高斯分布也可以表达成指数族分布的形式，此时$\mu\in\mathbb{R}^2$变为一个由$\mu,\ \sigma$表示的向量。为了将带有$\sigma^2$的高斯分布纳入GLM，我们可以给出更一般的指数族定义：

$$p(y;\eta,\tau)=b(a,\tau)\exp\left(\frac{\eta^TT(y)-a(\eta)}{c(\tau)}\right)$$

上式中的$\tau$称为**离散参数（dispersion parameter）**，而高斯分布对应的$c(\tau)=\sigma^2$。不过，对于我们简化的高斯分布，并不需要这种更一般的表达。）

$$\begin{align}p(y;\mu)&=\frac{1}{\sqrt{2\pi}}\exp\left(-\frac{1}{2}(y-\mu)^2\right)\\&=\frac{1}{\sqrt{2\pi}}\exp\left(-\frac{1}{2}y^2\right)\cdot\exp\left(\mu y-\frac{1}{2}\mu^2\right)\end{align}$$

于是，将高斯分布写为指数族形式有：

$$\begin{align}\eta&=\mu\\T(y)&=y\\a(\eta)&=\frac{\mu^2}{2}=\frac{\eta^2}{2}\\b(y)&=\frac{1}{\sqrt{2\pi}}\exp\left(\frac{-y^2}{2}\right)\end{align}$$

指数族当然还有很多别的成员，比如：多项分布（multinomial，我们将在后面介绍）、泊松分布（Poisson，用于计数数据建模）、伽马分布和指数分布（ the gamma and the exponential，用于非负的连续随机变量建模，比如时间间隔）、贝塔分布和狄利克雷分布（ the beta and the Dirichlet，对于概率的分布）等等。后面将介绍一种常规方法，来建立对于给定了$x,\theta$的来自上述任意分布的$y$的模型。